In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import mean_squared_error, roc_auc_score
from scipy.integrate import trapz
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtuples as tt
from pycox.models import CoxCC
from pycox.evaluation import EvalSurv
from survival_evaluation import d_calibration, l1
import statistics
from pycox.models import LogisticHazard, PMF, DeepHitSingle, CoxPH, MTLR
from sksurv.metrics import cumulative_dynamic_auc
import warnings
warnings.filterwarnings('ignore')

RuntimeError: module was compiled against NumPy C-API version 0x10 (NumPy 1.23) but the running NumPy has C-API version 0xf. Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem.

# CNN Attention Model for Battery RUL Prediction

This notebook implements a CNN-based model with attention mechanism for predicting the Remaining Useful Life (RUL) of NASA battery dataset using survival analysis.

In [2]:
class AttentionLayer(nn.Module):
    """Self-attention layer for feature importance weighting"""
    def __init__(self, input_dim, attention_dim=64):
        super(AttentionLayer, self).__init__()
        self.attention_dim = attention_dim
        self.W = nn.Linear(input_dim, attention_dim)
        self.V = nn.Linear(attention_dim, 1)
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        # x shape: (batch_size, seq_len, input_dim)
        # Compute attention scores
        attention_scores = self.V(self.tanh(self.W(x)))  # (batch_size, seq_len, 1)
        attention_weights = self.softmax(attention_scores.squeeze(-1))  # (batch_size, seq_len)
        
        # Apply attention weights
        weighted_features = x * attention_weights.unsqueeze(-1)  # (batch_size, seq_len, input_dim)
        attended_output = torch.sum(weighted_features, dim=1)  # (batch_size, input_dim)
        
        return attended_output, attention_weights

In [3]:
class CNNAttentionNet(nn.Module):
    """CNN with Attention mechanism for battery RUL prediction"""
    def __init__(self, input_features=15, num_filters=64, kernel_size=3, 
                 attention_dim=64, hidden_dims=[128, 64], output_dim=10, dropout=0.1):
        super(CNNAttentionNet, self).__init__()
        
        # Reshape input to create sequence dimension for CNN
        # We'll treat each feature as a time step
        
        # 1D CNN layers
        self.conv1 = nn.Conv1d(1, num_filters, kernel_size=kernel_size, padding=kernel_size//2)
        self.conv2 = nn.Conv1d(num_filters, num_filters*2, kernel_size=kernel_size, padding=kernel_size//2)
        self.conv3 = nn.Conv1d(num_filters*2, num_filters, kernel_size=kernel_size, padding=kernel_size//2)
        
        self.batch_norm1 = nn.BatchNorm1d(num_filters)
        self.batch_norm2 = nn.BatchNorm1d(num_filters*2)
        self.batch_norm3 = nn.BatchNorm1d(num_filters)
        
        # Attention mechanism
        self.attention = AttentionLayer(num_filters, attention_dim)
        
        # Fully connected layers
        layers = []
        prev_dim = num_filters
        
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.ReLU(),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(dropout)
            ])
            prev_dim = hidden_dim
            
        layers.append(nn.Linear(prev_dim, output_dim))
        self.fc_layers = nn.Sequential(*layers)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        batch_size = x.size(0)
        
        # Reshape input for 1D convolution: (batch_size, 1, input_features)
        x = x.unsqueeze(1)  # Add channel dimension
        
        # 1D CNN layers
        x = F.relu(self.batch_norm1(self.conv1(x)))
        x = self.dropout(x)
        
        x = F.relu(self.batch_norm2(self.conv2(x)))
        x = self.dropout(x)
        
        x = F.relu(self.batch_norm3(self.conv3(x)))
        x = self.dropout(x)
        
        # Transpose for attention: (batch_size, seq_len, features)
        x = x.transpose(1, 2)  # (batch_size, seq_len, num_filters)
        
        # Apply attention
        attended_features, attention_weights = self.attention(x)
        
        # Final classification/regression layers
        output = self.fc_layers(attended_features)
        
        return output

In [4]:
# Load and prepare data
path = './datacsv/battery_signature_features.csv'
D = pd.read_csv(path)

x_cols = D.columns[-15:].tolist()  # s_0 to s_14
event_col = ['event']  
time_col = ['time']   

D = D[x_cols + event_col + time_col]

print(f"Dataset shape: {D.shape}")
print(f"Feature columns: {x_cols}")
print(f"Event distribution: {D['event'].value_counts().to_dict()}")

Dataset shape: (2769, 17)
Feature columns: ['s_0', 's_1', 's_2', 's_3', 's_4', 's_5', 's_6', 's_7', 's_8', 's_9', 's_10', 's_11', 's_12', 's_13', 's_14']
Event distribution: {0: 1739, 1: 1030}


In [5]:
def train_val_test_stratified_split(df, stratify_colname='y', frac_train=0.6, frac_val=0.15, frac_test=0.25, random_state=None):
    if frac_train + frac_val + frac_test != 1.0:
        raise ValueError('fractions %f, %f, %f do not add up to 1.0' % (frac_train, frac_val, frac_test))
    if stratify_colname not in df.columns:
        raise ValueError('%s is not a column in the dataframe' % (stratify_colname))
    X = df # Contains all columns.
    y = df[[stratify_colname]] # Dataframe of just the column on which to stratify.
    df_train, df_temp, y_train, y_temp = train_test_split(X, y, stratify=y, test_size=(1.0 - frac_train), random_state=random_state)
    relative_frac_test = frac_test / (frac_val + frac_test)
    if relative_frac_test == 1.0:
        df_val, df_test, y_val, y_test = [], df_temp, [], y_temp
    else:
        df_val, df_test, y_val, y_test = train_test_split(df_temp, y_temp, stratify=y_temp, test_size=relative_frac_test, random_state=random_state)
    assert len(df) == len(df_train) + len(df_val) + len(df_test)
    return df_train, df_val, df_test

In [6]:
# Quick test of the CNN Attention model architecture
import torch

# Create a sample input to test the model
sample_input = torch.randn(32, 15)  # batch_size=32, features=15
print(f"Sample input shape: {sample_input.shape}")

# Initialize the CNN Attention network
test_net = CNNAttentionNet(
    input_features=15,
    num_filters=32,
    kernel_size=3,
    attention_dim=32,
    hidden_dims=[64, 32],
    output_dim=10,  # MTLR output dimensions
    dropout=0.1
)

print(f"Model architecture:")
print(test_net)

# Test forward pass
with torch.no_grad():
    output = test_net(sample_input)
    print(f"Output shape: {output.shape}")
    print(f"Model successfully processes input from {sample_input.shape} to {output.shape}")

# Count parameters
total_params = sum(p.numel() for p in test_net.parameters())
trainable_params = sum(p.numel() for p in test_net.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

Sample input shape: torch.Size([32, 15])
Model architecture:
CNNAttentionNet(
  (conv1): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3): Conv1d(64, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (batch_norm1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (attention): AttentionLayer(
    (W): Linear(in_features=32, out_features=32, bias=True)
    (V): Linear(in_features=32, out_features=1, bias=True)
    (tanh): Tanh()
    (softmax): Softmax(dim=1)
  )
  (fc_layers): Sequential(
    (0): Linear(in_features=32, out_features=64, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.1, 

In [7]:
# Initialize results storage
CI = []
IBS = []
L1_hinge = []
L1_margin = []
AUC_scores = []
n_exp = 30  # Number of experiments

d = D.copy()
cols_standardize = x_cols.copy()

print(f"Running {n_exp} experiments with CNN Attention model...")

Running 30 experiments with CNN Attention model...


In [8]:
import torch

print(torch.cuda.is_available())


True


In [9]:
for i in range(n_exp):
    print(f"\nExperiment {i+1}/{n_exp}")
    
    # Split data
    df_train, df_val, df_test = train_val_test_stratified_split(
        d, 'event', frac_train=0.8, frac_val=0.05, frac_test=0.15, random_state=10+i)
    
    # Standardize features
    standardize = [([col], StandardScaler()) for col in cols_standardize]
    x_mapper = DataFrameMapper(standardize)
    
    x_train = x_mapper.fit_transform(df_train).astype('float32')
    x_val = x_mapper.transform(df_val).astype('float32')
    x_test = x_mapper.transform(df_test).astype('float32')
    
    # Prepare survival data for MTLR
    get_target = lambda df: (df['time'].values, df['event'].values)
    num_durations = 10
    labtrans = MTLR.label_transform(num_durations)
    y_train = labtrans.fit_transform(*get_target(df_train))
    y_val = labtrans.transform(*get_target(df_val))
    
    val = (x_val, y_val)
    durations_test, events_test = get_target(df_test)
    
    # Create CNN Attention model
    in_features = x_train.shape[1]  # 15 features
    out_features = labtrans.out_features
    
    # Define CNN Attention network
    net = CNNAttentionNet(
        input_features=in_features,
        num_filters=32,
        kernel_size=3,
        attention_dim=32,
        hidden_dims=[64, 32],
        output_dim=out_features,
        dropout=0.1
    )
    
    # Use MTLR with our custom CNN network
    model = MTLR(net, tt.optim.Adam, duration_index=labtrans.cuts)
    
    # Find optimal learning rate
    batch_size = 128  # Smaller batch size for CNN
    lr_finder = model.lr_finder(x_train, y_train, batch_size, tolerance=6)
    best_lr = lr_finder.get_best_lr()
    model.optimizer.set_lr(best_lr * 0.1)  # Conservative learning rate
    
    # Train model
    epochs = 200
    callbacks = [tt.cb.EarlyStopping(patience=20)]
    
    try:
        model.fit(x_train, y_train, batch_size, epochs, callbacks, val_data=val, verbose=0)
        
        # Generate predictions
        surv = model.interpolate(10).predict_surv_df(x_test)
        
        # Save survival predictions
        surv_df = pd.DataFrame(surv)
        surv_df.index.name = 'time'
        surv_df.columns.name = 'survival_function'
        surv_df.to_csv(f"./datacsv/test-data/discharge_CNN_Attention_exp_{i+1}.csv", index=True)
        
        # Calculate survival predictions for L1 metrics
        survival_predictions = pd.Series(trapz(surv.values.T, surv.index), index=df_test.index)
        l1_hinge = l1(df_test.time, df_test.event, survival_predictions, l1_type='hinge')
        l1_margin = l1(df_test.time, df_test.event, survival_predictions, 
                      df_train.time, df_train.event, l1_type='margin')
        
        # Evaluation metrics
        ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
        c_index = ev.concordance_td('antolini')
        
        time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
        brier = ev.integrated_brier_score(time_grid)
        
        # AUC calculation
        quantiles = np.sort(df_test['time'].unique())
        labels_train = np.array([(e, t) for e, t in zip(df_train['event'], df_train['time'])], 
                               dtype=[('event', 'bool'), ('time', 'float')])
        labels_test = np.array([(e, t) for e, t in zip(df_test['event'], df_test['time'])], 
                              dtype=[('event', 'bool'), ('time', 'float')])
        
        auc_scores = []
        for eval_time in quantiles:
            try:
                interp_time_index = np.argmin(np.abs(eval_time - surv.index.values))
                surv_values_at_eval_time = surv.iloc[interp_time_index].values
                estimated_risks = 1 - surv_values_at_eval_time
                
                if np.min(estimated_risks) == np.max(estimated_risks):
                    continue
                    
                auc = cumulative_dynamic_auc(labels_train, labels_test, estimated_risks, times=[eval_time])[0][0]
                if not np.isnan(auc) and not np.isinf(auc):
                    auc_scores.append(auc)
            except Exception as e:
                continue
        
        # Store results
        AUC_scores.append(np.mean(auc_scores) if auc_scores else 0.5)
        CI.append(c_index)
        IBS.append(brier)
        L1_hinge.append(l1_hinge)
        L1_margin.append(l1_margin)
        
        print(f"CI: {c_index:.3f}, IBS: {brier:.3f}, AUC: {np.mean(auc_scores) if auc_scores else 0.5:.3f}")
        
    except Exception as e:
        print(f"Experiment {i+1} failed: {str(e)}")
        # Add default values for failed experiments
        AUC_scores.append(0.5)
        CI.append(0.5)
        IBS.append(1.0)
        L1_hinge.append(1.0)
        L1_margin.append(1.0)


Experiment 1/30
CI: 0.994, IBS: 0.008, AUC: 0.996

Experiment 2/30
CI: 0.993, IBS: 0.007, AUC: 0.998

Experiment 3/30
CI: 0.993, IBS: 0.008, AUC: 0.999

Experiment 4/30
CI: 0.993, IBS: 0.009, AUC: 0.998

Experiment 5/30
CI: 0.996, IBS: 0.008, AUC: 1.000

Experiment 6/30
CI: 0.995, IBS: 0.007, AUC: 0.999

Experiment 7/30
CI: 0.996, IBS: 0.007, AUC: 0.999

Experiment 8/30
CI: 0.993, IBS: 0.008, AUC: 0.998

Experiment 9/30
CI: 0.996, IBS: 0.007, AUC: 0.999

Experiment 10/30
CI: 0.992, IBS: 0.007, AUC: 0.998

Experiment 11/30
CI: 0.994, IBS: 0.008, AUC: 0.997

Experiment 12/30
CI: 0.994, IBS: 0.007, AUC: 0.999

Experiment 13/30
CI: 0.994, IBS: 0.007, AUC: 0.998

Experiment 14/30
CI: 0.995, IBS: 0.007, AUC: 0.999

Experiment 15/30
CI: 0.994, IBS: 0.007, AUC: 0.994

Experiment 16/30
CI: 0.995, IBS: 0.007, AUC: 0.998

Experiment 17/30
CI: 0.994, IBS: 0.008, AUC: 0.999

Experiment 18/30
CI: 0.995, IBS: 0.007, AUC: 0.998

Experiment 19/30
CI: 0.995, IBS: 0.007, AUC: 0.997

Experiment 20/30
CI:

In [10]:
# Calculate and display final results
def safe_stat(data):
    return round(statistics.mean(data), 3), round(statistics.stdev(data), 3) if len(data) > 1 else (0.0, 0.0)

auc_mean, auc_std = safe_stat(AUC_scores)
ci_mean, ci_std = safe_stat(CI)
ibs_mean, ibs_std = safe_stat(IBS)
l1h_mean, l1h_std = safe_stat(L1_hinge)
l1m_mean, l1m_std = safe_stat(L1_margin)

print("\n" + "="*60)
print("CNN ATTENTION MODEL RESULTS")
print("="*60)
print(f'Concordance Index (CI): {ci_mean} ± {ci_std}')
print(f'Integrated Brier Score (IBS): {ibs_mean} ± {ibs_std}')
print(f'L1 Hinge Loss: {l1h_mean} ± {l1h_std}')
print(f'L1 Margin Loss: {l1m_mean} ± {l1m_std}')
print(f'Area Under Curve (AUC): {auc_mean} ± {auc_std}')
print("="*60)

# Save results to CSV
results_df = pd.DataFrame({
    'Model': ['CNN_Attention'] * len(CI),
    'Experiment': range(1, len(CI) + 1),
    'CI': CI,
    'IBS': IBS,
    'L1_Hinge': L1_hinge,
    'L1_Margin': L1_margin,
    'AUC': AUC_scores
})

results_df.to_csv('./datacsv/cnn_attention_model_results.csv', index=False)
print(f"\nResults saved to: ./datacsv/cnn_attention_model_results.csv")


CNN ATTENTION MODEL RESULTS
Concordance Index (CI): 0.994 ± 0.001
Integrated Brier Score (IBS): 0.007 ± 0.001
L1 Hinge Loss: 24.333 ± 1.922
L1 Margin Loss: 1459.016 ± 13.844
Area Under Curve (AUC): 0.998 ± 0.001

Results saved to: ./datacsv/cnn_attention_model_results.csv


In [11]:
# Calibration analysis on final model
if len(CI) > 0:
    # Use the last successful model for calibration analysis
    try:
        # Re-run final model to get calibration data
        df_train, df_val, df_test = train_val_test_stratified_split(
            d, 'event', frac_train=0.8, frac_val=0.05, frac_test=0.15, random_state=10)
        
        standardize = [([col], StandardScaler()) for col in cols_standardize]
        x_mapper = DataFrameMapper(standardize)
        
        x_train = x_mapper.fit_transform(df_train).astype('float32')
        x_val = x_mapper.transform(df_val).astype('float32')
        x_test = x_mapper.transform(df_test).astype('float32')
        
        get_target = lambda df: (df['time'].values, df['event'].values)
        labtrans = MTLR.label_transform(10)
        y_train = labtrans.fit_transform(*get_target(df_train))
        y_val = labtrans.transform(*get_target(df_val))
        
        net = CNNAttentionNet(
            input_features=x_train.shape[1],
            num_filters=32,
            kernel_size=3,
            attention_dim=32,
            hidden_dims=[64, 32],
            output_dim=labtrans.out_features,
            dropout=0.1
        )
        
        model = MTLR(net, tt.optim.Adam, duration_index=labtrans.cuts)
        model.optimizer.set_lr(0.001)
        
        val = (x_val, y_val)
        callbacks = [tt.cb.EarlyStopping(patience=20)]
        model.fit(x_train, y_train, 128, 200, callbacks, val_data=val, verbose=0)
        
        surv = model.interpolate(10).predict_surv_df(x_test)
        
        # Calibration analysis
        calibration_results = d_calibration(df_test.event, surv.iloc[6])
        
        print("\nCALIBRATION ANALYSIS:")
        print(f"D-calibration p-value: {round(calibration_results['p_value'], 3)}")
        print("\nBin proportions:")
        for i, prop in enumerate(calibration_results['bin_proportions']):
            print(f"  Bin {i+1}: {prop:.3f}")
        
    except Exception as e:
        print(f"Calibration analysis failed: {str(e)}")


CALIBRATION ANALYSIS:
D-calibration p-value: 0.0

Bin proportions:
  Bin 1: 0.064
  Bin 2: 0.064
  Bin 3: 0.064
  Bin 4: 0.064
  Bin 5: 0.096
  Bin 6: 0.062
  Bin 7: 0.062
  Bin 8: 0.064
  Bin 9: 0.062
  Bin 10: 0.399


## Model Architecture Summary

The CNN Attention model consists of:

1. **1D Convolutional Layers**: Three conv1d layers with increasing then decreasing filter sizes (32→64→32) to extract local patterns from the 15 battery signature features

2. **Attention Mechanism**: Self-attention layer that computes importance weights for each feature, allowing the model to focus on the most relevant battery characteristics

3. **Fully Connected Layers**: Dense layers (64→32→output) with batch normalization and dropout for final survival prediction

4. **Survival Integration**: The model is wrapped with MTLR (Multi-Task Logistic Regression) for survival analysis compatible with censored data

### Key Features:
- **Feature Attention**: Automatically learns which battery signature features are most important
- **Temporal Patterns**: CNN layers capture local relationships between features
- **Regularization**: Batch normalization and dropout prevent overfitting
- **Survival Analysis**: Compatible with time-to-event data with censoring